In [1]:
import time
import numpy as np
from qboard.cimsim.cimsim import CIMSim
from qboard.cimsim import tuner
import qboard
import qboard.cache
from qboard.translators import markowitz
from qboard import qubo

import os
os.chdir('/home/koritskiy/rqc/Markowitz_partitioning')
os.getcwd()
from modules.partitioning import Partitioning

In [2]:
np.random.seed(4)
# Make console print look better
np.set_printoptions(precision=3,  # Digits after point
                    linewidth=170,  # Length of the line
                    suppress=True)  # Always fixed point notation (not scientific)


In [23]:
block_dim = 344


# define the file path
def path(name):
    return f"/home/koritskiy/rqc/Markowitz_partitioning/data/test_matrices/{block_dim}/{name}{block_dim}.csv"


# download matrices
block_mat = np.genfromtxt(path("block_mat"), delimiter=",")
mix_permutation_mat = np.genfromtxt(path("mix_permutation_mat"), delimiter=",")
mixed_mat = np.genfromtxt(path("mixed_mat"), delimiter=",")

# Create object of partitioning task
part = Partitioning(mixed_mat, theta=50)
bqm = part.bqm

In [28]:
Q = part.qubo_mat
h, J = qubo.toising(Q)
solutions = qboard.cache.Solutions()
# if (h, J) in solutions:
#     spins_ising = [1 if _ else -1 for _ in solutions[h, J]]
#     energy_ising = qubo.ienergy(h, J, spins_ising)
# else:
params = qboard.cache.Parameters()
if not params[h, J]:
    params[h, J] = tuner.optimize(h, J)
cimsim = CIMSim(J, h.reshape(-1, 1), device='cpu')
cimsim.set_params({'c_th':1., 'zeta':1., 'init_coupling':0.3, 'final_coupling':1., 'N':1000, 'attempt_num':10000, **params[h, J]})
(spins_ising, energy_ising, c_current, c_evol) = cimsim.find_opt()
solutions[h, J] = spins_ising
print('Im done')

In [29]:
solution = np.array([int((i+1)/2) for i in spins_ising])
part.permutation_mat = part.list_to_mat(solution)
print("Solution permutation matrix:")
print(part.permutation_mat)
print("\n")

# Check, whether solution matrix is permutation one
if not part.permutation_check(part.permutation_mat):
    print('\033[93m' + "Solution is not permutation matrix. " +
          "Trying to fix it" + '\033[0m')
    new_solution_permutation_mat = part.to_permutation(part.permutation_mat)
    if part.permutation_check(new_solution_permutation_mat):
        print('\033[93m' + "Success!" + '\033[0m')
        part.permutation_mat = new_solution_permutation_mat
    print("Solution matrix after fixing:")
    print(part.permutation_mat)

part.ordered_mat = part.permute(part.permutation_mat, part.mixed_mat)
print("Ordered matrix:")
print(part.ordered_mat)
print("\n")

Solution permutation matrix:
[[0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0]]


Solution is not permutation matrix. Trying to fix it
Number of wrong lines:
2
Success!
Solution matrix after fixing:
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
Ordered matrix:
[[0.926 0.377 0.582 0.    0.    0.    0.    0.    0.761 0.    0.   ]
 [0.377 0.809 0.602 0.    0.    0.    0.    0.    0.465 0.    0.   ]
 [0.582 0.602 0.812 0.   

In [45]:
a = 5
print(f"Hello {a}")

Hello 5
